In [ ]:
import xml.etree.ElementTree as etree
import codecs
import csv
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import math
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [2]:
FILENAME_WIKI = 'enwiki-20061130-pages-articles.xml'
WIKI_PAGES = 'wikipedia_pages.csv'
DISAMBIGUATION_PAGES = 'disambiguation.csv'
ENCODING = "utf-8"

In [3]:
# egy XML elem nevéből eltávolítja a namespace előtagot
def strip_tag_name(t):

    # a tag lekérése az elem objektumból
    t = elem.tag
    
    # megkeresi a jobb oldali legutolsó kapcsos zárójelet
    idx = t.rfind("}")
    
    # csak a namespace utáni részt tartja meg
    if idx != -1:
        t = t[idx + 1:]
    
    return t

In [ ]:
# megnyitjuk a két CSV fájlt írásra
# az egyik fájl a rendes Wikipedia oldalaknak van
# a másik a disambiguation oldalaknak
with codecs.open(WIKI_PAGES, "w", ENCODING) as wikiPage, \
    codecs.open(DISAMBIGUATION_PAGES, "w", ENCODING) as disambiguationPage:
    
    # QUOTE_MINIMAL azt jelenti, hogy csak akkor rak idézőjeleket a mezők köré, ha szükséges
    wikiPageWriter = csv.writer(wikiPage, quoting=csv.QUOTE_MINIMAL)
    
    # az első sorban fejléc: title és text
    wikiPageWriter.writerow(['title', 'text'])
    
    disambiguationPageWriter = csv.writer(disambiguationPage, quoting=csv.QUOTE_MINIMAL)
    
    disambiguationPageWriter.writerow(['title', 'text'])
    
    
    # XML fájl feldolgozása eseményalapon (iteratív olvasás, nem egyszerre az egész fájlba memóriába!).
    for event, elem in etree.iterparse(FILENAME_WIKI, events=('start', 'end')):
        
        # megtisztítjuk az elem nevét
        tname = strip_tag_name(elem.tag)
        
        # ha az esemény típusa start (tehát egy elem kezdetén járunk)
        if event == 'start':
            # ha az elem neve title, akkor eltároljuk a címet
            if tname == 'title':
                title = elem.text
            
            # ha az elem neve text, tehát a cikk szövege
            elif tname == 'text':
                # ellenőrizzük, hogy a szöveg és a cím nem üres
                if elem.text != None and title != None and 'disambiguation' in title:
                    # ha a cím tartalmazza a disambiguation szót, akkor egyértelműsítő oldalnak tekintjük
                    disambiguationPageWriter.writerow([title, elem.text])
                
      
                elif elem.text != None and title != None and len(elem.text) > 1024 and "Image:" not in title and 'This page has been deleted' not in elem.text:
                    wikiPageWriter.writerow([title, elem.text])
        
        # felszabadítjuk az aktuális elem memóriáját, hogy ne fogyjon el a memória a nagy XML feldolgozásakor
        elem.clear()


In [5]:
pages = pd.read_csv('wikipedia_pages.csv')
disambiguation = pd.read_csv('disambiguation.csv')

In [ ]:
# létrehozunk egy szótárt, amelynek kulcsai a pages DataFrame title oszlopából származó címek kisbetűs változatai
# minden kulcshoz az érték 1 lesz

keywords_titles = {}
for i in range(0, len(pages)):
    keywords_titles[str(pages['title'][i]).lower()] = 1


In [ ]:
titles_roots = {}


for keyword in keywords_titles:
    
    # a címet szóköz mentén feldaraboljuk szavakra, minden szóra alkalmazzuk a szótövezőt (stemmer),
    root = ' '.join(map(str, [stemmer.stem(word) for word in keyword.split(' ')]))

    if root != keyword:
        titles_roots[root] = keywords_titles[keyword]


In [10]:
# létrehozunk egy szótárat (dictionary-t), amely a stopwords-kat tartalmazza
# minden stop-szót kulcsként tárolunk a szótárban, és az értékét 1-re állítjuk.

stopwords_dict = {k: 1 for k in stopwords.words('english')}


In [11]:
# a függvény visszaad egy listát, amely az adott szövegből előállított n-grammákat tartalmazza
def get_ngrams(filtered_article, n):
    
    # Minden szó a szöveg egy lehetséges eltolásával kerül párba. Az n elem minden egyes pozícióra generál egy n-grammot
    ngrams = zip(*[filtered_article[i:] for i in range(n)])
    
    # az n-grammák listájának elkészítése
    return [" ".join(ngram) for ngram in ngrams]


In [ ]:
nr_of_documents_with_word_as_keyword = {}
nr_of_documents_with_word = {}


# végigmegyünk az összes dokumentum szövegén (feltételezve, hogy ez egy lista vagy sorozat)
for text in pages['text']:
        
    words_this_text = {}
    tokenized_article = word_tokenize(text)

    # csak azokat a szavakat tartjuk meg, amelyek betűkből állnak
    # és nem szerepelnek a stopwordsben
    filtered_article = [w.lower() for w in tokenized_article if w.isalpha() and stopwords_dict.get(w.lower(), 0) == 0]
    
    # ha a szó szótöve szerepel a címek gyökerei között, vagy kulcsszóként ismert
    for word in filtered_article:
        # hozzáadjuk a szót a dokumentumban talált releváns szavakhoz
        if titles_roots.get(stemmer.stem(word), 0) == 1 or keywords_titles.get(word, 0) == 1:
            words_this_text[word] = words_this_text.get(word, 0) + 1
    
    
    for n in range(2, 4):
        ngrams = get_ngrams(filtered_article, n)
        for ngram in ngrams:
            root = ' '.join(map(str, [stemmer.stem(word) for word in ngram.split(' ')]))
            # ha ez a szótövek kombinációja vagy maga az ngram szerepel a kulcsszók között
            if titles_roots.get(root, 0) == 1 or keywords_titles.get(ngram, 0) == 1:
                # hozzáadjuk a dokumentumban talált releváns szókapcsolatokhoz
                words_this_text[ngram] = words_this_text.get(ngram, 0) + 1
        
        # kiválasztjuk a leggyakoribb (felső 6%-nyi) szavakat, mint potenciális kulcsszavakat
    keywords_to_consider = sorted(words_this_text.items(), key=lambda kv: kv[1], reverse=True)[:int(len(words_this_text) * 0.06)] 
    
    # növeljük az összes dokumentumban előforduló szavak számlálóját
    for word in words_this_text.keys():
        nr_of_documents_with_word[word] = nr_of_documents_with_word.get(word, 0) + 1
        
    # növeljük azoknak a szavaknak a számlálóját, amelyek ebben a dokumentumban kulcsszóként jelentek meg
    for word in keywords_to_consider:
        nr_of_documents_with_word_as_keyword[word[0]] = nr_of_documents_with_word_as_keyword.get(word[0], 0) + 1


In [14]:
with open("nr_of_documents_with_word_as_keyword.json", "w") as file:
    json.dump(nr_of_documents_with_word_as_keyword, file, indent=4)

with open("nr_of_documents_with_word.json", "w") as file:
    json.dump(nr_of_documents_with_word , file, indent=4) 

In [15]:
titles_roots_lists = {}

for keyword in keywords_titles:
    root = ' '.join(map(str, [stemmer.stem(word) for word in keyword.split(' ')]))

    # csak akkor dolgozzuk fel, ha a root nem egyezik az eredeti címmel
    if root != keyword:
        # ha ez a root már létezik és a keyword még nincs a listában, hozzáadjuk
        if root in titles_roots_lists:
            if keyword not in titles_roots_lists[root]:
                titles_roots_lists[root].append(keyword)
        else:
            # ha még nem szerepel a root, új listával indítjuk
            titles_roots_lists[root] = [keyword]


In [16]:
with open('nr_of_documents_with_word.json') as json_file:
    nr_of_documents_with_word = json.load(json_file)

with open('nr_of_documents_with_word_as_keyword.json') as json_file:
    nr_of_documents_with_word_as_keyword = json.load(json_file)

In [17]:
keys = list(nr_of_documents_with_word.keys())

for key in keys:
    # ha egy szó legfeljebb 5 dokumentumban fordul elő, eltávolítjuk
    if nr_of_documents_with_word[key] <= 5:
        nr_of_documents_with_word.pop(key)


In [18]:
keys = list(nr_of_documents_with_word_as_keyword.keys())

for key in keys:
    if nr_of_documents_with_word_as_keyword[key] <= 5:
        nr_of_documents_with_word_as_keyword.pop(key)


In [22]:
# szótár létrehozása, amelyben a dokumentumokban szereplő szavak rootjai vannak eltárolva
# a kulcs: root, az érték: annak maximális előfordulása dokumentumkulcsszóként
nr_of_documents_with_word_roots = {}

# végigmegyünk az összes szón (vagy n-gramon)
for keyword in nr_of_documents_with_word:

    # az adott szó(n-gram) szótövét képezzük úgy, hogy minden szót külön stemmelünk, majd visszarakjuk őket egy sztringgé
    root = ' '.join(map(str, [stemmer.stem(word) for word in keyword.split(' ')]))

    # csak akkor folytatjuk, ha a szótő különbözik az eredeti szótól
    if root != keyword:
        # a szótőhöz tartozó érték a korábban eltárolt érték és az aktuális keyword-hoz tartozó érték maximuma lesz
        # így mindig a legnagyobb dokumentumszámot tartjuk meg, ahol a root előfordul
        nr_of_documents_with_word_roots[root] = max(
            nr_of_documents_with_word_roots.get(root, 0),  # ha már szerepel, lekérjük az eddigi értéket
            nr_of_documents_with_word[keyword]             # összevetjük a jelenlegi kulcsszó előfordulásával
        )


In [23]:
nr_of_documents_with_word_as_keyword_roots = {}

for keyword in nr_of_documents_with_word_as_keyword:
    root = ' '.join(map(str, [stemmer.stem(word) for word in keyword.split(' ')]))
    if root != keyword:
        nr_of_documents_with_word_as_keyword_roots[root] = max(nr_of_documents_with_word_as_keyword_roots.get(root, 0), nr_of_documents_with_word_as_keyword[keyword])

In [24]:
# ez a függvény visszaadja, hogy egy adott szó (vagy kifejezés) hány dokumentumban szerepelt
# először pontos egyezést keres, ha nincs, akkor a szótövére próbál keresni

def n(w):
    # megnézzük, hogy a szó (w) pontosan hány dokumentumban szerepel
    nr = nr_of_documents_with_word.get(w, 0)
    if nr != 0:
        return nr

    # ha a pontos szó nem szerepel, megnézzük a szótövéhez tartozó dokumentumszámot
    nr = nr_of_documents_with_word_roots.get(w, 0)
    if nr != 0:
        return nr
    
    return 0


In [ ]:
# a függvény célja, hogy visszaadja a szövegben található szavak és n-gramok előfordulásait
# szavanként számolja meg, hogy hányszor fordulnak elő, valamint a 2-gramokat és 3-gramokat is
def get_occurences_of_word(text):    
    # szótár, amely a szavakat és azok előfordulási számát tartalmazza a szövegben
    words_this_text = {}
    
    # a szöveget tokenizáljuk (szavakra bontjuk), majd kisbetűssé alakítjuk
    # csak azokat a szavakat tartjuk meg, amelyek betűkből állnak és nem szerepelnek a stop szavak között
    filtered_article = [w.lower() for w in word_tokenize(text) if w.isalpha() and stopwords_dict.get(w.lower(), 0) == 0]
    
    for word in filtered_article:
        # ha a szó már szerepel a szótárban, növeljük a számlálót, ha nem, akkor 1-re állítjuk
        words_this_text[word] = words_this_text.get(word, 0) + 1
    
    for n in range(2, 4):
        # n-gramokat generálunk a szövegből (2-gramok és 3-gramok)
        ngrams = get_ngrams(filtered_article, n)
        for ngram in ngrams:
            # A generált n-gramokat is hozzáadjuk a szótárhoz, és növeljük azok előfordulását
            words_this_text[ngram] = words_this_text.get(ngram, 0) + 1
    
    return words_this_text


In [26]:
# a szavak előfordulásainak értékei alapján kiválassza a linkelendő kulcsszavakat
def words_to_link(scores):
    # lekérjük azokat a kulcsszavakat, amelyek az előfordulások szerint a legfontosabbak
    # a 'scores' egy szótár, ahol a kulcsok a szavak, az értékek pedig az előfordulásuk számát jelzik
    # az első 6%-át vesszük a legnagyobb előfordulási számmal rendelkező szavaknak
    keywords_to_consider = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)[:int(len(scores) * 0.06)]
    
    link = []

    for ngram in [item[0] for item in keywords_to_consider]:
        # a kulcsszót (ngram) szótövezés alá vetjük, hogy egységesebbek legyenek a formák
        root = ' '.join(map(str, [stemmer.stem(word) for word in ngram.split(' ')]))
        
        # ha a szótő vagy a kulcsszó szerepel a címekhez rendelt szótárban, akkor ezt hozzáadjuk a linkekhez
        if titles_roots_lists.get(root, 0) != 0 or keywords_titles.get(ngram, 0) != 0:
            link.append(ngram)
            
    # töröljük a redundáns elemeket (pl. ha egy n-gram több szót tartalmaz, és az egyes szavak is szerepelnek a listában)
    for item in link:
        # csak azokat az n-gramokat nézzük, amelyek több mint egy szóból állnak
        if len(item.split(' ')) >= 2:
            for word in item.split(' '):
                # ha a szó szerepel az n-gramok között, akkor eltávolítjuk
                if word in link:
                    link.remove(word)
        
    return link


Tf.idf

In [27]:
N = len(pages)

In [ ]:
# a tf kiszámolja egy szó gyakoriságát egy adott szövegben
# 'w' a szó, amit keresünk, words pedig egy szótár, amely tartalmazza a szó előfordulásait
def tf(w, words):
    # a szó előfordulása a szövegben: ha a szó szerepel, visszaadjuk az előfordulás számát,
    # ha nem szerepel, akkor 0-t adunk vissza.
    return words.get(w, 0)

# a tf-idf (Term Frequency-Inverse Document Frequency) függvény célja, hogy kiszámolja
# egy szó súlyát, figyelembe véve a szó előfordulását egy szövegben (tf) és annak ritkaságát
# az összes dokumentumban (idf)
def tf_idf(w, words):
    # 'words' egy szótár, amely a szövegben található szavakat és azok előfordulásait tartalmazza
    # az 'n(w)' a szó előfordulásának számát adja vissza az összes dokumentumban
    nr = n(w)
    
    # Ha a szó előfordul legalább egyszer (nr != 0), akkor kiszámoljuk a tf-idf értékét
    if nr != 0:
        # a tf-idf értéke: tf(w) * log(N / n(w)), ahol N az összes dokumentumok száma
        # n(w) pedig a szó előfordulásainak száma
        return tf(w, words) * math.log(N / n(w))
    else:
        return 0


In [29]:
# meghatározza a szövegben található kulcsszavakat a tf-idf alapján, 
# majd kiválassza azokat a szavakat, amelyek linkelhetők.
def get_words_to_link_tf_idf(text):
    # először is meghatározzuk a szó előfordulásokat a szövegben
    # a get_occurences_of_word függvény egy szótárat ad vissza, amely tartalmazza a szövegben
    # található szavakat és azok előfordulásait
    words_in_this_text = get_occurences_of_word(text)
    
    # kiszámítjuk a tf-idf értékeket a szavakra
    # A tf-idf_w egy szótár, amely tartalmazza a szavakat és azok tf-idf értékeit
    tf_idf_w = {}

    for w in words_in_this_text.keys():
        # a tf-idf érték kiszámolása a szó számára a get_occurences_of_word által visszaadott szótár
        # és a tf_idf függvény használatával
        tf_idf_w[w] = tf_idf(w, words_in_this_text)
    
    # visszaadjuk a legfontosabb kulcsszavakat
    # a words_to_link függvény fogja meghatározni, hogy mely szavakat kell linkelni
    return words_to_link(tf_idf_w)


Keyphraseness

In [ ]:
# ha a szó nem szerepel közvetlenül, akkor a szótövének előfordulásait is figyelembe veszi
def n_keyword(w):
    # ellenőrizzük, hogy a szó hány dokumentumban szerepel kulcsszóként
    # ha a szó szerepel a kulcsszavak között, akkor visszaadjuk az előfordulás számát
    nr = nr_of_documents_with_word_as_keyword.get(w, 0)
    if nr != 0:
        return nr
    
    # ha a szó nem szerepel közvetlenül, akkor megnézzük a szótövének előfordulását
    # hogy a szótöve szerepel-e kulcsszóként más dokumentumokban
    nr = nr_of_documents_with_word_as_keyword_roots.get(w, 0)
    if nr != 0:
        return nr

    return 0


In [31]:
# a kulcsszóra vonatkozó arányt adja vissza, azaz a kulcsszó gyakoriságát a szó összes előfordulásához képest
def keyphraseness_score(w):
    # ha a szó előfordulásának száma nem 0
    # akkor kiszámítjuk a kulcsszóra vonatkozó arányt.
    if n(w) != 0:
        # a kulcsszóhoz tartozó előfordulások számát elosztjuk a szó összes előfordulásával.
        # ez adja a kulcsszóra vonatkozó arányt
        return n_keyword(w) / n(w)
    else:
        return 0


In [32]:
# a legfontosabb kulcsszavakat határozza meg, amelyeket linkelni érdemes.
def get_words_to_link_keyphraseness(text):
    # először kellenek a szövegben előforduló szavakat és azok előfordulásai
    words_in_this_text = get_occurences_of_word(text)
    
    # Létrehozzuk a kulcsszóra vonatkozó értékeléseket
    keyphraseness_scores = {}

    # minden szóra kiszámítjuk a kulcsszó értékelést
    for w in words_in_this_text.keys():
        keyphraseness_scores[w] = keyphraseness_score(w)
    
    # a kiszámított kulcsszó értékelések alapján meghatározzuk a linkelhető szavakat
    return words_to_link(keyphraseness_scores)


Összehasonlítás

In [33]:
text = "Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from data. These systems improve their performance over time without being" \
"explicitly programmed. Applications include image recognition, natural language processing, and recommendation systems. It is widely used in industries such as healthcare, finance" \
"and marketing."


keyphraseness_words = get_words_to_link_keyphraseness(text)

tf_idf_words = get_words_to_link_tf_idf(text)


# közös kulcsszavak keresése a két lista között
common_keywords = set(keyphraseness_words).intersection(tf_idf_words)

# közös kulcsszavak számossága
common_count = len(common_keywords)

# kulcsszavak száma mindkét listában
keyphraseness_count = len(set(keyphraseness_words))
tf_idf_count = len(set(tf_idf_words))

# arány számítása
common_ratio_keyphraseness = common_count / keyphraseness_count if keyphraseness_count != 0 else 0
common_ratio_tf_idf = common_count / tf_idf_count if tf_idf_count != 0 else 0

# a közös kulcsszavak aránya
common_ratio = common_count / max(keyphraseness_count, tf_idf_count) if max(keyphraseness_count, tf_idf_count) != 0 else 0

common_ratio


0.25